# Preprocessing & Splits

## 🎯 Concept Primer

Preprocessing transforms raw features into ML-ready format. **Critical rule:** Fit scalers/encoders ONLY on training data, then transform val/test to prevent leakage.

### Preprocessing Steps
1. **Encode categoricals** — One-Hot or Ordinal encoding
2. **Scale continuous** — StandardScaler (mean=0, std=1) or MinMaxScaler (0-1)
3. **Split data** — Train (70%) / Val (15%) / Test (15%), stratified by target
4. **Handle imbalance** — Class weights, oversampling, or threshold tuning

**Expected outputs:** X_train, y_train, X_val, y_val, X_test, y_test

## 📋 Objectives

By the end of this notebook, you will:
1. Encode categorical features (One-Hot or Ordinal)
2. Scale continuous features using StandardScaler
3. Split into train/val/test (70/15/15 stratified)
4. Choose an imbalance handling strategy
5. Verify shapes and dtypes

## ✅ Acceptance Criteria

You'll know you're done when:
- [ ] All categoricals encoded
- [ ] Continuous features scaled
- [ ] Data split into train/val/test
- [ ] Imbalance strategy chosen and documented
- [ ] Shapes printed: X_train.shape, y_train.shape, etc.
- [ ] No leakage (transformers fit only on train)

## 🔧 Setup

In [32]:
# TODO 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import torch
from sklearn.preprocessing import OrdinalEncoder


df = pd.read_csv("../../../datasets/diabetes_BRFSS2015.csv")
df.columns = df.columns.str.lower().str.replace(' ', '_')
numeric_cols = ['bmi', 'genhlth', 'menthlth', 'physhlth']

df.head()

,highbp,highchol,cholcheck,bmi,smoker,stroke,heartdiseaseorattack,physactivity,fruits,veggies,hvyalcoholconsump,anyhealthcare,nodocbccost,genhlth,menthlth,physhlth,diffwalk,sex,age,education,income,diabetes
0,Yes,Yes,Yes,40.0,Yes,No,No,No,No,Yes,No,Yes,No,5.0,18.0,15.0,Yes,Female,60-64,Grade 12 or GED (High school graduate),"Less than $20,000",No Diabetes
1,No,No,No,25.0,Yes,No,No,Yes,No,No,No,No,Yes,3.0,0.0,0.0,No,Female,50-54,College 4+ years (College graduate),"Less than $10,000",No Diabetes
2,Yes,Yes,Yes,28.0,No,No,No,No,Yes,No,No,Yes,Yes,5.0,30.0,30.0,Yes,Female,60-64,Grade 12 or GED (High school graduate),"$75,000 or more",No Diabetes
3,Yes,No,Yes,27.0,No,No,No,Yes,Yes,Yes,No,Yes,No,2.0,0.0,0.0,No,Female,70-74,Grades 9-11 (Some high school),"Less than $50,000",No Diabetes
4,Yes,Yes,Yes,24.0,No,No,No,Yes,Yes,Yes,No,Yes,No,2.0,3.0,0.0,No,Female,70-74,College 1-3 years (Some college/technical school),"Less than $25,000",No Diabetes


## 🏷️ Separate Features and Target

### TODO 2: Split data into X and y

**Expected:**
- X: All columns except `diabetes_binary`
- y: Only `diabetes_binary`

**Shapes:** X will be (N, D) where D = number of features

In [33]:
# TODO 2: Separate features and target
diabetes_map = {'No Diabetes': 0, 'Prediabetes': 1, 'Diabetes': 2}
df['diabetes_trinary'] = df['diabetes'].map(diabetes_map)

# 3. Then create X and y
X = df.drop(['diabetes', 'diabetes_trinary'], axis=1)  # drop both original and encoded
y = df['diabetes_trinary']

# 4. Verify encoding worked
print("Target encoding check:")
print(y.value_counts())  # Should show 0, 1, 2
print(y.dtype)   

Target encoding check:
diabetes_trinary
0    213703
2     35346
1      4631
Name: count, dtype: int64
int64


## 📊 Handle Imbalance

### TODO 3: Choose imbalance strategy

**Options:**
1. **Class weights** — Weight loss function by class frequency
2. **Oversampling** — SMOTE or RandomOverSampler (on train only!)
3. **Threshold tuning** — Adjust decision threshold at inference

**Decision:** Choose one and document why in reflection.

**Check imbalance:**
```python
y.value_counts(normalize=True)
```

In [34]:
# TODO 3: Check imbalance
print(y.value_counts(normalize=True))

# If using class weights:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weight_dict = {0: class_weights[0], 1: class_weights[1], 2: class_weights[2]}
print(class_weight_dict)

diabetes_trinary
0    0.842412
2    0.139333
1    0.018255
Name: proportion, dtype: float64
{0: 0.3956893445576337, 1: 18.259555171669184, 2: 2.3923499122955922}


## 🔄 Encode Categoricals

### TODO 4: Apply One-Hot encoding

**Columns to encode:** Binary features (already 0/1) and ordinal features

**Options:**
- OneHotEncoder: Creates separate columns for each category
- Keep it simple: Most columns are already numeric!

**Expected:** After encoding, all features should be numeric

In [35]:
# TODO 4: Encode categoricals (if needed)

# Do this BEFORE any encoding/scaling
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Encode on ALL splits (deterministic, no fitting needed)
binary_cols = ['highbp', 'highchol', 'cholcheck', 'smoker', 'stroke', 
               'heartdiseaseorattack', 'physactivity', 'fruits', 'veggies',
               'hvyalcoholconsump', 'anyhealthcare', 'nodocbccost', 'diffwalk']

X_train[binary_cols] = X_train[binary_cols].replace({'Yes': 1, 'No': 0})
X_val[binary_cols] = X_val[binary_cols].replace({'Yes': 1, 'No': 0})
X_test[binary_cols] = X_test[binary_cols].replace({'Yes': 1, 'No': 0})

# Verify
print("Binary encoding check:")
print(X_train[binary_cols].head())
print(X_train['highbp'].unique())  # Should see [0, 1]

ordinal_cols = ['age', 'education', 'income']

# Fix the syntax and fit
ordinal_enc.fit(X_train[ordinal_cols])  # Double brackets!

# Transform all splits
X_train[ordinal_cols] = ordinal_enc.transform(X_train[ordinal_cols])
X_val[ordinal_cols] = ordinal_enc.transform(X_val[ordinal_cols])
X_test[ordinal_cols] = ordinal_enc.transform(X_test[ordinal_cols])

# Verify
print("\nOrdinal encoding check:")
print(X_train[ordinal_cols].head())
print("Age range:", X_train['age'].min(), "to", X_train['age'].max())  # Should be 0 to 12


# Option 1: Binary encode (Male=1, Female=0)
# Since only 2 categories, no need for one-hot
X_train['sex'] = X_train['sex'].map({'Male': 1, 'Female': 0})
X_val['sex'] = X_val['sex'].map({'Male': 1, 'Female': 0})
X_test['sex'] = X_test['sex'].map({'Male': 1, 'Female': 0})

# Option 2: One-hot encode (if you prefer)
# from sklearn.preprocessing import OneHotEncoder
# But I recommend Option 1 for binary

# Verify
print("\nSex encoding check:")
print(X_train['sex'].unique())  # Should see [0, 1]

from sklearn.preprocessing import StandardScaler

numeric_cols = ['bmi', 'genhlth', 'menthlth', 'physhlth']
scaler = StandardScaler()

# Fit on train only
scaler.fit(X_train[numeric_cols])

# Transform all splits
X_train[numeric_cols] = scaler.transform(X_train[numeric_cols])
X_val[numeric_cols] = scaler.transform(X_val[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Verify
print("\nScaling check:")
print(X_train[numeric_cols].describe())  # Mean should be ~0, std should be ~1

print("\n" + "="*80)
print("FINAL PREPROCESSING CHECK")
print("="*80)

print(f"\nShapes:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"X_test: {X_test.shape}")

print(f"\nData types:")
print(X_train.dtypes.value_counts())

print(f"\nClass distribution preserved:")
print(y_train.value_counts(normalize=True))
print(y_val.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

print(f"\nNo missing values:")
print(f"Train: {X_train.isnull().sum().sum()}")
print(f"Val: {X_val.isnull().sum().sum()}")
print(f"Test: {X_test.isnull().sum().sum()}")

print("\n✅ Preprocessing complete!")

/var/folders/cd/2c6x3jgj47j_fwl7231ts7m00000gn/T/ipykernel_37892/3608101978.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train[binary_cols] = X_train[binary_cols].replace({'Yes': 1, 'No': 0})
/var/folders/cd/2c6x3jgj47j_fwl7231ts7m00000gn/T/ipykernel_37892/3608101978.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_val[binary_cols] = X_val[binary_cols].replace({'Yes': 1, 'No': 0})
/var/folders/cd/2c6x3jgj47j_fwl7231ts7m00000gn/T/ipykernel_37892/3608101978.py:18: FutureWarning: Downcasting behavior in `replace` is d

Binary encoding check:
        highbp  highchol  cholcheck  smoker  stroke  heartdiseaseorattack  physactivity  fruits  veggies  hvyalcoholconsump  anyhealthcare  nodocbccost  diffwalk
2725         1         1          1       1       0                     0             1       1        1                  0              1            0         1
119890       1         1          1       1       0                     0             1       0        1                  0              1            0         0
148149       0         0          1       0       0                     0             1       1        1                  0              1            0         0
91717        1         1          1       1       0                     1             0       1        1                  0              1            0         0
102495       0         0          1       1       0                     0             1       0        1                  1              1            1         0
[1 0]

## ⚖️ Scale Continuous Features

### TODO 5: Apply StandardScaler

**Fit on train only!** Then transform val/test.

**Features to scale:** BMI, MentHlth, PhysHlth, Age (if numeric)

**Process:**
1. Split data first
2. Fit scaler on X_train
3. Transform X_train, X_val, X_test

In [36]:
# TODO 5: Scale features
# Split first
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Then scale
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_val_scaled = scaler.transform(X_val)
# X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames
# X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
# X_val = pd.DataFrame(X_val_scaled, columns=X_val.columns)
# X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)

## ✅ Verify Splits

### TODO 6: Print shapes and check dtypes

**Expected outputs:**
- X_train: (N_train, D)
- y_train: (N_train,)
- Similar for val and test
- All should be numeric (float or int)

In [37]:
# TODO 6: Verify splits
print(f"Train: X={X_train.shape}, y={y_train.shape}")
print(f"Val: X={X_val.shape}, y={y_val.shape}")
print(f"Test: X={X_test.shape}, y={y_test.shape}")
print(f"\nDtypes:\n{X_train.dtypes}")

Train: X=(177576, 21), y=(177576,)
Val: X=(38052, 21), y=(38052,)
Test: X=(38052, 21), y=(38052,)

Dtypes:
highbp                    int64
highchol                  int64
cholcheck                 int64
bmi                     float64
smoker                    int64
stroke                    int64
heartdiseaseorattack      int64
physactivity              int64
fruits                    int64
veggies                   int64
hvyalcoholconsump         int64
anyhealthcare             int64
nodocbccost               int64
genhlth                 float64
menthlth                float64
physhlth                float64
diffwalk                  int64
sex                       int64
age                     float64
education               float64
income                  float64
dtype: object


In [38]:
import pickle

data_dict = {
    'X_train': X_train,
    'X_val': X_val,
    'X_test': X_test,
    'y_train': y_train,
    'y_val': y_val,
    'y_test': y_test,
    'class_weights': class_weight_dict
}

import pickle
import os

output_dir = '../preprocessed_data'
os.makedirs(output_dir, exist_ok=True)

with open(os.path.join(output_dir, 'preprocessed_train_test_val.pkl'), 'wb') as f:
    pickle.dump(data_dict, f)

## 🤔 Reflection

1. **Imbalance strategy:** Which did you choose? Why?
2. **Scaler choice:** StandardScaler vs MinMaxScaler? Why?
3. **Split strategy:** Why 70/15/15? Is test set large enough?
4. **Leakage check:** Are you sure no val/test info leaked into training?

**Your reflection:**

### 1. Imbalance Strategy

**Chosen:** Class weights (balanced approach)

**Computed weights:**
- Class 0 (No Diabetes): 0.396 - down-weighted (84% of data)
- Class 1 (Prediabetes): 18.26 - heavily up-weighted (2% of data)
- Class 2 (Diabetes): 2.39 - moderately up-weighted (14% of data)

**Rationale:**
- Severe class imbalance (84% / 14% / 2%) requires intervention
- Without weights, model would predict "No Diabetes" for everything (84% accuracy but useless)
- Class weights force model to pay attention to minority classes during training
- Loss function penalizes prediabetes misclassifications 18× more than no diabetes misclassifications
- Combined with stratified sampling for clean workflow

**Trade-offs:**
- Prediabetes weight (18.26) is very high and may cause training instability
- Alternative would be capping max weight at 10 or using SMOTE
- Starting with balanced weights as baseline; will monitor training behavior

### 2. Scaler Choice

**Chosen:** StandardScaler (z-score normalization)

**How it works:**
- Transforms features to mean ≈ 0, standard deviation ≈ 1
- Formula: `(x - mean) / std`

**Why StandardScaler:**
- Robust to outliers compared to MinMaxScaler
- Works well with features that have different scales (BMI 12-60 vs MentHlth 0-30)
- Better for skewed distributions (many zeros in mental/physical health)
- Neural networks and gradient-based algorithms prefer standardized inputs
- Tree-based models less affected but standardization helps convergence

**Why NOT MinMaxScaler:**
- Scales to [0,1] range - sensitive to outliers
- Even after BMI capping, distribution still skewed
- StandardScaler preserves distribution shape while normalizing scale

**Applied to:** Only numeric columns (bmi, genhlth, menthlth, physhlth)
**Not applied to:** Binary (already 0/1) and ordinal (already meaningful integers)

### 3. Split Strategy

**Chosen:** 70/15/15 (train/validation/test)

**Breakdown:**
- Training: 70% (~177,576 samples) - used to fit models
- Validation: 15% (~38,052 samples) - used for hyperparameter tuning
- Test: 15% (~38,052 samples) - held out for final evaluation

**Is test set large enough?**
Yes, for several reasons:
- 38,052 samples provides statistically reliable metrics
- Even prediabetes class (2%) has ~760 test samples
- Sufficient to measure per-class performance with confidence
- Large enough to detect meaningful differences between models

**Why this split:**
- 70% train: Large enough for stable model training, especially with 253K total samples
- 15% val: Adequate for hyperparameter search without overfitting to validation set
- 15% test: True unseen data for honest final evaluation

**Alternative considered:** 80/10/10 would maximize training data but reduce validation/test reliability

**Stratification:** All splits use `stratify=y` to preserve class proportions (84/14/2) in each split

### 4. Leakage Check

**Leakage prevention measures:**

✅ **Split FIRST, then transform**
- Data split before any encoding or scaling
- Ensures val/test are truly "unseen" during preprocessing

✅ **Encoders fit on training only**
- Binary encoding: Deterministic (Yes→1, No→0), no learning needed
- Ordinal encoder: `.fit(X_train[ordinal_cols])` then `.transform()` on val/test
- Training data determines category ordering

✅ **Scaler fit on training only**
- `scaler.fit(X_train[numeric_cols])` computes mean/std from training
- Same parameters applied to val/test via `.transform()`
- Val/test never influence scaling parameters

✅ **Stratified sampling**
- Maintains class distribution across all splits
- No information leak, just proportional representation

**Verification:**
- Class proportions match across train/val/test (all ~84/14/2)
- No overlapping samples between splits (verified by indices)
- Transformer parameters only computed from training data

**Conclusion:** No data leakage. Preprocessing workflow follows best practices.

## 📌 Summary

✅ **Encoded:** Categoricals converted to numeric  
✅ **Scaled:** Continuous features standardized  
✅ **Split:** Train/val/test created  
✅ **Balanced:** Imbalance strategy applied  
✅ **Ready for next step:** Train baseline models

**Next notebook:** `06_baselines_logreg_rf.ipynb`